In [1]:
import numpy as np
import pandas as pd    

In [2]:
def relu(x):
    return np.maximum(0,x)

In [3]:
def deriv_relu(x):
    der=list()
    der.clear()
    for val in x:
        if val>0:
            der.append(1)
        else:
            der.append(0)
    return der

In [4]:
def initialize(x, neuron_num1, neuron_num2):
    np.random.seed(12345)
    w1=2*np.random.random((len(x.columns), neuron_num1))-1
    w2=2*np.random.random((neuron_num1, neuron_num2))-1
    w3=2*np.random.random((neuron_num2, 1))-1
    bias1=2*np.random.random()-1
    bias2=2*np.random.random()-1
    bias3=2*np.random.random()-1
    return w1, w2, w3, bias1, bias2, bias3

In [5]:
def forward_prop(w1, w2, w3, bias1, bias2, bias3, x):
    x=x.reshape(13,1)
    z1=np.dot(w1.T,x)+bias1
    out1=relu(z1)
    z2=np.dot(w2.T,out1)+bias2
    out2=relu(z2) 
    z3=np.dot(w3.T,out2)+bias3
    out3=relu(z3)
    return z1, out1, z2, out2, z3, out3 

In [6]:
def back_prop(w1, w2, w3, bias1, bias2, bias3, x, y, z1, out1, z2, out2, z3, out3, lr):
    x=x.reshape(13,1)
    
    err_3=2*(np.subtract(out3.sum(axis=0), y))
    dummy_3=np.array(deriv_relu(z3)).reshape(len(z3),1)
    dc_dw3=lr*np.dot(out2,dummy_3)*err_3
    w3=np.subtract(w3,dc_dw3)
    dc_db3=lr*err_3
    bias3-=dc_db3
    
    
    err_2=2*(np.subtract(out2.sum(axis=0), y)) # karenin turevi oldugu icin 2*(y_pred-y_act)
    dummy_2=np.array(deriv_relu(z2)).reshape(len(z2),1)
    dc_dw2=lr*np.dot(out1,dummy_2.T)*err_2
    w2=np.subtract(w2,dc_dw2)
    dc_db2=lr*err_2
    bias2-=dc_db2

    err_1= 2*(np.subtract(out1.sum(axis=0), y))
    dummy_1=np.array(deriv_relu(z1)).reshape(len(z1),1)
    dc_dw1=lr*np.dot(x,dummy_1.T)*err_1
    w1=np.subtract(w1,dc_dw1)
    dc_db1=lr*err_1
    bias1-=dc_db1
    return w1, w2, w3, bias1, bias2, bias3

In [7]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [8]:
columns=("CRIM","ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV")

In [9]:
ds=pd.read_csv("/Users/ecemdenizbabaoglan/Desktop/TOBBETU/yap470/housing.csv", sep='\s+', engine='python', names=columns)

In [10]:
target=ds.loc[:,'MEDV'].copy()
data=ds.drop('MEDV', axis=1).copy()

In [11]:
neuron_num1=7
neuron_num2=7

In [12]:
w1, w2, w3, bias1, bias2, bias3 = initialize(data, neuron_num1, neuron_num2)

In [13]:
data_normalized = ((data - data.mean()) / data.std(ddof=0)).to_numpy()
target=target.to_numpy()

In [14]:
cutoff = int(len(data) * 0.8)
x_train, x_test = data_normalized[:cutoff], data_normalized[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [15]:
epochs=100
lr=0.0001

In [16]:
train_loss_epoch=list()
test_loss_epoch=list()

In [17]:
for e in range(epochs):
    loss_train=0
    loss_test=0
    for i in range(len(y_train)):   
        z1, out1, z2, out2, z3, out3= forward_prop(w1, w2, w3, bias1, bias2, bias3, x_train[i])
        w1, w2, w3, bias1, bias2, bias3 = back_prop(w1, w2, w3, bias1, bias2, bias3, x_train[i], y_train[i], z1,
                                                    out1, z2, out2, z3, out3, lr)
        loss_train+=np.square(y_train[i]-(out3.sum(axis=0)))
        
    train_loss_epoch.append(loss_train/(len(y_train)))
         
    for i in range(len(y_test)):
        z_test1, out_test1, z_test2, out_test2, z_test3, out_test3=forward_prop(w1, w2, w3, bias1, bias2, bias3, x_test[i])
        loss_test+=np.square(y_test[i]-(out_test3.sum(axis=0)))

    test_loss_epoch.append(loss_test/(len(y_test)))

In [22]:
print(test_loss_epoch)

[array([184.93480392]), array([54.89597727]), array([46.64908999]), array([41.54408493]), array([37.25447415]), array([34.56096027]), array([32.48450095]), array([31.21621985]), array([30.04371557]), array([29.03763162]), array([27.95276319]), array([27.12046716]), array([26.60708779]), array([26.21800994]), array([25.74652102]), array([25.22387118]), array([24.85360127]), array([24.59489657]), array([24.31380105]), array([23.95392756]), array([23.76776115]), array([23.45059691]), array([23.13734204]), array([22.75299516]), array([22.38259488]), array([21.96631951]), array([21.54820028]), array([21.15623299]), array([20.71702485]), array([20.36176466]), array([20.0343004]), array([19.80558092]), array([19.53236088]), array([19.353936]), array([19.13052153]), array([18.92860766]), array([18.80607189]), array([18.80618005]), array([18.6458804]), array([18.55487235]), array([18.49725936]), array([18.45602725]), array([18.39033528]), array([18.32964872]), array([18.28693721]), array([18.58

In [18]:
#for i in range(len(y_train)):
   # print(y_train[i], output[i].sum(axis=0))
#    loss+=(y_train[i]-(output3[i].sum(axis=0)))
#print(loss/(len(y_train)))

In [19]:
#for i in range(len(y_train)):
#    print(y_train[i], output3[i])

In [20]:
#out_test=list()
#for i in range(len(y_test)): 
#    z_test1, out_test1, z_test2, out_test2, z_test3, out_test3=forward_prop(w1, w2, w3, bias1, bias2, bias3, x_test[i])
#    out_test.append(out_test3)
#    loss_test+=(y_test[i]-(out_test[i].sum(axis=0)))
#print(loss_test/(len(y_test)))

In [21]:
#for i in range(len(y_test)):
#    print(y_test[i], out_test[i])